In [5]:
import os
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
os.chdir('/content/drive/MyDrive/surya/NLP_Assignment/Programming Assignment - 3')

Mounted at /content/drive


In [2]:
!pip install transformers datasets torch evaluate scikit-learn nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 23.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl 

#Data Preparation
Let's start by loading the train.en (English) and train.or (Oriya) datasets and splitting them into train, validation, and test sets.

Load and Split Dataset
We will load the dataset from the file paths provided and split it into training, validation, and test sets.

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the English and Hindi data
train_en_file = "/content/drive/MyDrive/surya/NLP_Assignment/Programming Assignment - 3/en-or/train.en"
train_hi_file = "/content/drive/MyDrive/surya/NLP_Assignment/Programming Assignment - 3/en-or/train.or"

# Initialize lists to store lines
en_texts = []
hi_texts = []

# Read English file line by line and store in list
with open(train_en_file, 'r', encoding='utf-8') as en_file:
    en_texts = en_file.readlines()

# Read Hindi file line by line and store in list
with open(train_hi_file, 'r', encoding='utf-8') as hi_file:
    hi_texts = hi_file.readlines()

# Create pandas DataFrames from the lists
train_en = pd.DataFrame(en_texts, columns=['en_text'])
train_hi = pd.DataFrame(hi_texts, columns=['hi_text'])

# Merge English and Hindi data based on line numbers
data = pd.concat([train_en, train_hi], axis=1)

# Sub-sampling: randomly select a fraction of the data
subsample_fraction = 0.1  # Adjust the fraction as needed (e.g., 0.1 for 10% of the data)
data = data.sample(frac=subsample_fraction, random_state=42)

# Split the data into train, validation, and test sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=42)

print(f"Training data: {len(train_data)} samples")
print(f"Validation data: {len(val_data)} samples")
print(f"Test data: {len(test_data)} samples")

# Display a few examples from the training data
print("Training example:")
print(train_data.head())


Training data: 63886 samples
Validation data: 15972 samples
Test data: 19965 samples
Training example:
                                                  en_text  \
257986  However, in face of a tremendous public respon...   
64239   And David said unto Achish, If I have now foun...   
232245  This city is popular with tourists in the wint...   
876064  In the past Sambalpur has been a great centre ...   
182929  Today I am proud when I hear that India's rela...   

                                                  hi_text  
257986  ତେବେ ଏଥିପ୍ରତି ଅଭୂତପୂର୍ବ ଜନସମର୍ଥନ ତଥା ଅଧିକରୁ ଅଧ...  
64239   "ଦାଉଦ ଅଖୀଶଙ୍କକ୍ସ୍ଟ କହିଲେ, ""ଯଦି ତୁମ୍ଭେ ମାେ ପ୍ର...  
232245  ଶୀତଦିନେ ଏହି ସହର ପର୍ଯ୍ୟଟକଙ୍କୁ ଲୋଭାନ୍ୱିତ କରିଥାଏ ।\n  
876064  ପୁରାକାଳରେ ସମ୍ବଲପୁର ହୀରା ବ୍ୟବସାୟର ପ୍ରଧାନ କେନ୍ଦ୍...  
182929  ଆଜି ମୋତେ ଏହା କହି ଗର୍ବ ଅନୁଭବ ହେଉଛି କି ଗଲ୍ଫ ସହିତ...  


#Tokenization
Let's define tokenizers for the three models (BART-base, T5-base, and a custom model using BERT-base). The tokenizers will be used to process the text data.

In [8]:
from transformers import BartTokenizer, T5Tokenizer, BertTokenizer

# Define the tokenizers
bart_tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
t5_tokenizer = T5Tokenizer.from_pretrained('t5-base')
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


#Training Functions
We will define functions for training and evaluating the models.

Train Model
Define a function to train a model using a given training and validation dataset.

In [9]:
import torch
from torch.utils.data import DataLoader
from transformers import AdamW, get_linear_schedule_with_warmup
from nltk.translate.bleu_score import sentence_bleu

def train_model(model, tokenizer, train_data, val_data, num_epochs=3, batch_size=8, learning_rate=5e-5, gradient_accumulation_steps=1):
    # Prepare data loaders
    def collate_fn(batch):
        inputs = [x[0] for x in batch]
        labels = [x[1] for x in batch]
        # Tokenize and pad inputs and labels
        tokenized_inputs = tokenizer(inputs, return_tensors='pt', padding=True, truncation=True, max_length=512)
        tokenized_labels = tokenizer(labels, return_tensors='pt', padding=True, truncation=True, max_length=512)
        return tokenized_inputs, tokenized_labels['input_ids']

    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

    # Define optimizer and scheduler
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    total_steps = len(train_loader) * num_epochs
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

    # Define the training loop
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for batch in train_loader:
            inputs, labels = batch
            inputs = {key: val.to(device) for key, val in inputs.items()}
            labels = labels.to(device)

            outputs = model(**inputs, labels=labels)

            # Check for loss
            assert hasattr(outputs, 'loss'), "Model output does not have a 'loss' attribute"
            loss = outputs.loss
            total_loss += loss.item()

            # Perform backward pass and optimize
            loss.backward()
            if (batch_idx + 1) % gradient_accumulation_steps == 0:
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()

            batch_idx += 1

        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(train_loader)}")

        # Evaluate on the validation set
        evaluate_model(model, tokenizer, val_loader)

    return model

# Evaluation function for the validation set
def evaluate_model(model, tokenizer, val_loader):
    model.eval()
    total_bleu_score = 0
    count = 0

    with torch.no_grad():
        for batch in val_loader:
            inputs, labels = batch
            inputs = {key: val.to(device) for key, val in inputs.items()}
            labels = labels.to(device)

            outputs = model.generate(**inputs)

            # Convert token IDs to text
            predicted_texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)
            reference_texts = tokenizer.batch_decode(labels, skip_special_tokens=True)

            # Compute BLEU score
            for pred, ref in zip(predicted_texts, reference_texts):
                total_bleu_score += sentence_bleu([ref.split()], pred.split())
                count += 1

    # Calculate average BLEU score
    avg_bleu_score = total_bleu_score / count if count > 0 else 0
    print(f"Validation BLEU Score: {avg_bleu_score:.4f}")


#Evaluate Model
Define a function to evaluate a trained model using the test dataset.

In [10]:
import evaluate
from transformers import pipeline

def evaluate_model(model, tokenizer, test_loader):
    # Define metrics
    bleu = evaluate.load("sacrebleu")
    rouge = evaluate.load("rouge")

    # Evaluate the model
    model.eval()
    references = []
    predictions = []

    with torch.no_grad():
        for batch in test_loader:
            inputs, labels = batch
            references += labels
            inputs = tokenizer(inputs, return_tensors='pt', padding=True, truncation=True, max_length=512)

            inputs = inputs.to(device)

            outputs = model.generate(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'])
            outputs_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)
            predictions += outputs_text

    # Calculate metrics
    bleu_score = bleu.compute(predictions=predictions, references=references)
    rouge_score = rouge.compute(predictions=predictions, references=references)

    print(f"BLEU Score: {bleu_score['score']}")
    print(f"ROUGE Score: {rouge_score}")


#Model Training and Evaluation
Let's train and evaluate the three models: BART-base, T5-base, and a custom BERT-based seq2seq model.

In [11]:
from transformers import BartForConditionalGeneration, T5ForConditionalGeneration, BertForSequenceClassification

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the models
bart_model = BartForConditionalGeneration.from_pretrained('facebook/bart-base').to(device)
t5_model = T5ForConditionalGeneration.from_pretrained('t5-base').to(device)
bert_model = BertForSequenceClassification.from_pretrained('bert-base-uncased').to(device)


print("Train data columns:", train_data.columns)
print("Validation data columns:", val_data.columns)
print("Test data columns:", test_data.columns)



# Train and evaluate BART-base
print("Training and evaluating BART-base...")
bart_model = train_model(bart_model, bart_tokenizer, train_data, val_data)
test_loader = DataLoader(test_data, batch_size=8, collate_fn=lambda x: (x[0], x[1]))
evaluate_model(bart_model, bart_tokenizer, test_loader)

# Train and evaluate T5-base
print("Training and evaluating T5-base...")
t5_model = train_model(t5_model, t5_tokenizer, train_data, val_data)
evaluate_model(t5_model, t5_tokenizer, test_loader)

# Train and evaluate BERT-base as a seq2seq model
print("Training and evaluating BERT-base...")
bert_model = train_model(bert_model, bert_tokenizer, train_data, val_data)
evaluate_model(bert_model, bert_tokenizer, test_loader)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Train data columns: Index(['en_text', 'hi_text'], dtype='object')
Validation data columns: Index(['en_text', 'hi_text'], dtype='object')
Test data columns: Index(['en_text', 'hi_text'], dtype='object')
Training and evaluating BART-base...


KeyError: 28399

In [4]:
import torch
from torch.utils.data import DataLoader
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import BartForConditionalGeneration, T5ForConditionalGeneration, BertForSequenceClassification
from transformers import BertTokenizer, BartTokenizer, T5Tokenizer
import pandas as pd

# Set the device for model training and evaluation
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the file paths
train_en_file = "/content/drive/MyDrive/surya/NLP_Assignment/Programming Assignment - 3/en-hi/train.en"
train_hi_file = "/content/drive/MyDrive/surya/NLP_Assignment/Programming Assignment - 3/en-hi/train.hi"

# Use the appropriate separator for your data files; the default is a comma (',')
train_en = pd.read_csv(train_en_file, header=None, names=['en_text'], sep=',', on_bad_lines='warn')
train_hi = pd.read_csv(train_hi_file, header=None, names=['hi_text'], sep=',', on_bad_lines='warn')


# Merge the data
data = pd.concat([train_en, train_hi], axis=1)

# Split the data into train, validation, and test sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=42)

print(f"Training data: {len(train_data)} samples")
print(f"Validation data: {len(val_data)} samples")
print(f"Test data: {len(test_data)} samples")

# Load tokenizers
bart_tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
t5_tokenizer = T5Tokenizer.from_pretrained('t5-base')
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Load models
bart_model = BartForConditionalGeneration.from_pretrained('facebook/bart-base').to(device)
t5_model = T5ForConditionalGeneration.from_pretrained('t5-base').to(device)
bert_model = BertForSequenceClassification.from_pretrained('bert-base-uncased').to(device)

# Collate function to process batches
def collate_fn(batch):
    input_texts, labels_texts = zip(*batch)
    # Convert input_texts and labels_texts to lists before using the tokenizer
    input_texts_list = list(input_texts)
    labels_texts_list = list(labels_texts)
    inputs = bart_tokenizer(input_texts_list, return_tensors='pt', padding=True, truncation=True, max_length=512)
    labels = bart_tokenizer(labels_texts_list, return_tensors='pt', padding=True, truncation=True, max_length=512)
    return inputs, labels

# Train model function
def train_model(model, tokenizer, train_data, val_data, num_epochs=3, batch_size=8, learning_rate=5e-5):
    # Prepare data loaders
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

    # Define optimizer and scheduler
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    total_steps = len(train_loader) * num_epochs
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

    # Define the training loop
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for batch in train_loader:
            try:
                inputs, labels = batch
                # Move tensors to device
                inputs = {k: v.to(device) for k, v in inputs.items()}
                labels = labels['input_ids'].to(device)

                # Forward pass
                outputs = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'], labels=labels)

                # Compute loss and backpropagate
                loss = outputs.loss
                total_loss += loss.item()
                loss.backward()

                # Optimizer step and zero gradients
                optimizer.step()
                optimizer.zero_grad()

                # Scheduler step
                scheduler.step()

            except Exception as e:
                print(f"Error during training: {e}")
                # Handle specific error cases as necessary

        print(f"Epoch {epoch + 1}/{num_epochs} Loss: {total_loss / len(train_loader)}")

        # Evaluate on validation set
        evaluate_model(model, tokenizer, val_loader)

    return model

# Evaluate model function
def evaluate_model(model, tokenizer, data_loader):
    model.eval()
    total_loss = 0
    for batch in data_loader:
        with torch.no_grad():
            try:
                inputs, labels = batch
                # Move tensors to device
                inputs = {k: v.to(device) for k, v in inputs.items()}
                labels = labels['input_ids'].to(device)

                # Forward pass
                outputs = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'], labels=labels)

                # Compute loss
                loss = outputs.loss
                total_loss += loss.item()

            except Exception as e:
                print(f"Error during evaluation: {e}")
                # Handle specific error cases as necessary

    print(f"Validation Loss: {total_loss / len(data_loader)}")

# Train and evaluate the BART-base model
print("Training and evaluating BART-base...")
train_model(bart_model, bart_tokenizer, train_data, val_data)
evaluate_model(bart_model, bart_tokenizer, test_data)


Streaming output truncated to the last 5000 lines.
Skipping line 9999365: expected 4 fields, saw 8
Skipping line 9999395: expected 4 fields, saw 6
Skipping line 9999435: expected 4 fields, saw 5
Skipping line 9999498: expected 4 fields, saw 6
Skipping line 9999503: expected 4 fields, saw 7
Skipping line 9999514: expected 4 fields, saw 6
Skipping line 9999532: expected 4 fields, saw 7
Skipping line 9999541: expected 4 fields, saw 6
Skipping line 9999557: expected 4 fields, saw 8
Skipping line 9999569: expected 4 fields, saw 5
Skipping line 9999619: expected 4 fields, saw 5
Skipping line 9999629: expected 4 fields, saw 13
Skipping line 9999630: expected 4 fields, saw 7
Skipping line 9999641: expected 4 fields, saw 6
Skipping line 9999647: expected 4 fields, saw 5
Skipping line 9999668: expected 4 fields, saw 6
Skipping line 9999689: expected 4 fields, saw 6
Skipping line 9999721: expected 4 fields, saw 5
Skipping line 9999727: expected 4 fields, saw 5
Skipping line 9999744: expected 4 fi

NotImplementedError: Can only union MultiIndex with MultiIndex or Index of tuples, try mi.to_flat_index().union(other) instead.